In [ ]:
def getPage(url):
  try:
    import urllib.request
    page = urllib.request.urlopen(url).read()
    page = page.decode("utf-8")
    return page
  except:
    return ""

def get_next_target(page):
    start_link = page.find('<a href=')
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote+1)
    url = page[start_quote + 1:end_quote]
    return url, end_quote

def get_all_links(page):
    links = []
    while True:
      url, endpos = get_next_target(page)
      if url:
        links.append(url)
        page = page[endpos:]
      else:
        break
    return links

def union(p,q):
  for e in q:
    if e not in p:
      p.append(e)

def add_to_index(index, keyword, url):
  if keyword in index:
    index[keyword].append(url)
  else:
    index[keyword] = [url]

def addPageToIndex(index, url, content):
  words = content.split()
  for word in words:
    add_to_index(index, word, url)

def crawlWeb(seed):
  tocrawl = [seed]
  crawled = []
  index = {}
  graph = {}

  while tocrawl:
    page = tocrawl.pop()
    if page not in crawled:
      content = getPage(page)
      addPageToIndex(index, page, content)
      outlinks = get_all_links(content)
      graph [page] = outlinks
      union(tocrawl, get_all_links(content))
      crawled.append(page)

  return index, graph

def computeRanks(graph):
  d = 0.8
  N = len(graph) # number of pages
  numloops = 10 # effects the accuracy
  ranks = {}

  for page in graph :
    ranks[page] = 1/N

  for i in range (0, numloops):
    newranks = {}
    for page in graph :
      newrank = (1-d)/N
      for node in graph :
        if page in graph[node]:
          newrank = newrank + d*(ranks[node]/len(graph[node]))
      newranks[page] = newrank
    ranks = newranks
  return newranks


index , graph = crawlWeb("https://searchengineplaces.com.tr/")
ranks = computeRanks(graph)
print(ranks)


{'https://searchengineplaces.com.tr/': 0.019999999999999997, 'http://www.searchengineplaces.com.tr/travel_guide.html': 0.14780429869056003, 'http://www.searchengineplaces.com.tr/seymarecommends.html': 0.073060618584064, 'http://www.searchengineplaces.com.tr/oktayrecommends.html': 0.073060618584064, 'http://www.searchengineplaces.com.tr/istanbul.html': 0.17460832634470402, 'http://www.searchengineplaces.com.tr/galata_tower.html': 0.09025810358272002, 'http://www.searchengineplaces.com.tr/maidens_tower.html': 0.09025810358272002, 'http://www.searchengineplaces.com.tr/konya.html': 0.073060618584064, 'http://www.searchengineplaces.com.tr/mevlana.html': 0.04928445079552, 'http://www.searchengineplaces.com.tr/ankara.html': 0.043776167788544}
